In [1]:
import nltk
import random
import pandas as pd
import numpy as np
from nltk.corpus import brown, reuters

In [2]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [3]:
df = pd.read_csv("/kaggle/input/truncated_sentences.csv")


In [4]:
df.head()

,truncated_sentence,genre
0,"If possible ,",religion
1,It was enough,romance
2,The school year,government
3,As a source,government
4,Members of The,government


In [5]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
login(user_secrets.get_secret("HUGGINGFACE_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
import re

#function to extract first sentence from the block of text generated / if no termination is found, the entire sentence is returned.
def sentence_extractor(text_to_extract):
  pattern = r'^[^.]*?(?:\.{1,3})(?=\s+[A-Z]|$)'
  match = re.search(pattern, text_to_extract)
  if match:
    return match.group()
  else:
    return text_to_extract

In [7]:
from datasets import Dataset


dataset = Dataset.from_pandas(df.drop(columns='genre'))

dataset

Dataset({
    features: ['truncated_sentence'],
    num_rows: 1500
})

In [8]:
def post_data_processing(file_name):
  print("I was called.")
  data = []
  for i in range(len(df)):
    for output in outputs[i]:
        data.append([df['truncated_sentence'][i], df['genre'][i], sentence_extractor(output['generated_text']), current_model])
  columns = ['truncated_sentence', 'genre', 'generated_sentence', 'model']
  dataset = pd.DataFrame(data, columns=columns)
  dataset.to_csv(file_name, index=False)

In [9]:
%%skip

import transformers
import torch
import re
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

current_model = 'microsoft/Phi-3.5-mini-instruct'
generator = pipeline("text-generation", model=current_model, device=0)

outputs = []
for out in tqdm(generator(KeyDataset(dataset, "truncated_sentence"), truncation=True, max_length=50, num_return_sequences=3, return_full_text=False, do_sample=True, top_k = 50, pad_token_id=generator.tokenizer.eos_token_id)):
    outputs.append(out)

In [10]:
%%skip

post_data_processing("phi.csv")

In [11]:
import transformers
import torch
import re
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

current_model = 'Qwen/Qwen2.5-1.5B-Instruct'
generator = pipeline("text-generation", model=current_model, device=0)

outputs = []
for out in tqdm(generator(KeyDataset(dataset, "truncated_sentence"), truncation=True, max_length=50, num_return_sequences=3, return_full_text=False, do_sample=True, top_k = 50, pad_token_id=generator.tokenizer.eos_token_id)):
    outputs.append(out)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

In [12]:
post_data_processing("qwen.csv")

I was called.
